# Example using EM
Expectation maximization (EM) helps generate the gaussian mixture guesses based on a guess for the size of the object. This assumes that the user already has an Observation. We will start by creating our fake observation from the Basic_fitting notebook.

In [2]:
# This block is copied from Basic_fitting. The comments
# are left in, but for additional details see that notebook.
# necessary imports
import ngmix
from ngmix.observation import Observation, ObsList, MultiBandObsList
from ngmix.fitting import LMSimple
import numpy
from numpy import array
from numpy.random import uniform as urand
# eps is a constant that we need and represents 1% accuracy
eps = 0.01
# Seed the RNG
numpy.random.seed(8381)
gal_jacob = ngmix.UnitJacobian(row = 16.0, col = 16.0)
psf_jacob = ngmix.UnitJacobian(row = 12.0, col = 12.0)
# Object is an exponential disk approximated by gaussians.
pars  = [0.0, 0.0, 0.2, -0.1, 16.0, 100.0]
gmix0 = ngmix.GMixModel(pars, "exp")
# PSF is a single gaussian
psf_pars = [0.0, 0.0, -0.03, 0.02, 4.0, 1.0]
psf_gmix = ngmix.GMixModel(psf_pars, "gauss")
# Convolve the two
gmix = gmix0.convolve(psf_gmix)
dimensions = [32, 32]
image0 = gmix.make_image(dimensions, npoints=10, jacobian=gal_jacob)
psf_dimensions = [24, 24]
psf_image = psf_gmix.make_image(psf_dimensions, npoints=10, jacobian=psf_jacob)
# Add noise to the galaxy image
sigma = 0.01
noise = numpy.random.normal(scale = sigma, size = image0.shape)
image = image0 + noise
# Make an observation of the psf image
psf_obs = Observation(psf_image, jacobian=psf_jacob)
# We use a 'simple' model fit with 6 parameters.
# For simplicity we will guess these parameters before pixelization
pfitter = LMSimple(psf_obs, "gauss")
guess = array(psf_pars)
guess[0] += urand(low=-eps, high=eps)
guess[1] += urand(low=-eps, high=eps)
guess[2] += urand(low=-eps, high=eps)
guess[3] += urand(low=-eps, high=eps)
guess[4] *= (1.0 + urand(low=-eps, high=eps))
guess[5] *= (1.0 + urand(low=-eps, high=eps))
# Kick off the fitter and get out the mixture of the fit
pfitter.go(guess)
psf_gmix_fit = pfitter.get_gmix()
# Set the mixture to the observation. This is needed for galaxy fitting later.
psf_obs.set_gmix(psf_gmix_fit)
weight = numpy.zeros(image.shape) + 1.0/sigma**2
obs = Observation(image, weight=weight, jacobian = gal_jacob, psf = psf_obs)
# Now we have an object.

In [3]:
# Import the EM runner object
from ngmix.bootstrap import EMRunner

In [4]:
# Define the number of gaussians to fit, the guess of the position,
# and the number of times to try with different guesses of T.
ngauss = 3
Tguess = 4.0
ntry   = 5

In [5]:
# Define the parameters for the runner's fit, and create the runner
# object. Then tell it to go and start the fit.
em_pars = {"maxiter": 1000, "tol": 1.0e-6}
runner = EMRunner(obs, Tguess, ngauss, em_pars)
runner.go(ntry = ntry)

In [10]:
# Get the fitter from the runner and its result, then print it
fitter = runner.get_fitter()
result = fitter.get_result()
print result

{'flags': 1, 'ntry': 5}


In [12]:
if result["flags"] != 0:
    print "an error occured with flags: %d"%result['flags']
else:

    # get a ngmix.GMix object
    gmix = runner.get_gmix()

    # get the full parametrization as an array.  Note EM does
    # not normalize the p values to give the total
    # flux
    #
    # [ p1, row1, col1, irr1, irc1, icc1,
    #   p2, row2, col2, irr2, irc2, icc2,
    #   p3, row3, col3, irr3, irc3, icc3 ]

    pars = gmix.get_full_pars()

an error occured with flags: 1
